<a href="https://colab.research.google.com/github/BianPaulParaguya/SPY-Monitor/blob/main/SPY_monitor_1hr_24h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install pandas
!pip install numpy


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output

def get_stock_data(ticker="SPY", period="1mo", interval="1h"):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period, interval=interval)
    return hist

def calculate_macd(data):
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def calculate_rsi(data, period=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_bollinger_bands(data, window=20):
    sma = data['Close'].rolling(window=window).mean()
    std = data['Close'].rolling(window=window).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, lower_band

def analyze_stock(ticker="SPY"):
    data = get_stock_data(ticker)
    macd, signal = calculate_macd(data)
    rsi = calculate_rsi(data)
    upper_band, lower_band = calculate_bollinger_bands(data)

    latest_macd = macd.iloc[-1]
    latest_signal = signal.iloc[-1]
    latest_rsi = rsi.iloc[-1]
    latest_upper_band = upper_band.iloc[-1]
    latest_lower_band = lower_band.iloc[-1]
    latest_close = data['Close'].iloc[-1]

    suggestion = "Hold"
    action = "None"

    if latest_macd > latest_signal and latest_rsi < 70 and latest_close < latest_upper_band:
        suggestion = "Potential Upside Swing"
        action = "Consider Buying"
    elif latest_macd < latest_signal and latest_rsi > 30 and latest_close > latest_lower_band:
        suggestion = "Potential Downside Swing"
        action = "Consider Selling"

    return {
        "MACD": latest_macd,
        "Signal": latest_signal,
        "RSI": latest_rsi,
        "Upper Band": latest_upper_band,
        "Lower Band": latest_lower_band,
        "Suggestion": suggestion,
        "Action": action,
        "Close": latest_close
    }

def track_stock(ticker="SPY", interval=3600):
    while True:
        data = []
        analysis = analyze_stock(ticker)
        data.append([
            ticker,
            analysis['MACD'],
            analysis['Signal'],
            analysis['RSI'],
            analysis['Upper Band'],
            analysis['Lower Band'],
            analysis['Suggestion'],
            analysis['Action'],
            analysis['Close']
        ])

        df = pd.DataFrame(data, columns=[
            'Ticker', 'MACD', 'Signal', 'RSI', 'Upper Band', 'Lower Band', 'Suggestion', 'Action', 'Close'
        ])

        clear_output(wait=True)
        display(df)
        time.sleep(interval)

ticker = "SPY"
interval = 30  # 1 hour interval
track_stock(ticker, interval)


,Ticker,MACD,Signal,RSI,Upper Band,Lower Band,Suggestion,Action,Close
0,SPY,0.715154,1.684913,38.251379,562.124086,555.775444,Hold,None,555.619995
